In [1]:
import os
import matplotlib.pyplot as plt
import glob
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
from rasterio.mask import mask
from rasterstats import zonal_stats
import fiona
import geopandas as gpd
import xarray as xr
import numpy as np
import rioxarray
import pandas as pd
import dask

# Set all nan values to np.nan for parameters (only need to run once)

In [ ]:
def reassign_mask_npnan(param_dirs):
    
    for folder in param_dirs:
        files=np.array(sorted(os.listdir(folder)))
        tifs = pd.Series(files).str.contains('.tif')
        files = files[tifs]
        print(folder)
        
        for filename in files:
            with rasterio.open(folder+filename, 'r+') as ds:
                print(filename)
                a = ds.read()# read all raster values
                a[a < 0 ] = np.nan  
                ds.write(a)

In [ ]:
reassign_mask_npnan(all_dirs)

# Create Land Cover Masks

In [ ]:
def create_land_mask(in_dir, out_dir, lc_num):
    files=np.array(sorted(os.listdir(in_dir)))
    
    for filename in files:
        base = os.path.splitext(filename)[0]
        print(base)
        rast = rasterio.open(in_dir+filename)
        meta = rast.meta
        meta.update(dtype = 'float32', nodata = -3.4e+38)
        rast.close()
        
        with rasterio.open(out_dir+base+'_' + str(lc_num) + '.tif', 'w', **meta) as dst:
            with rasterio.open(in_dir+filename) as src:
                data = src.read()
                data = data.astype(np.float32)
                data[data!=lc_num] = np.nan
                data[data==lc_num] = 1
                dst.write(data)

In [ ]:
landcover_in_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/Original/'
landcover_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/WoodySavannas_Mask/'
lc_class = 8


In [ ]:
create_land_mask(landcover_in_dir, landcover_out_dir, lc_class)

In [2]:
#ALL DIRECTORIES USED FOR PARAMETER RASTER CALCULATION

grasslands_mask_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/Grasslands_Mask/'
croplands_mask_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/Croplands_Mask/'
cropnatveg_mask_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/CropNatVeg_Mask/'
savannas_mask_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/Savannas_Mask/'
woodysavannas_mask_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/WoodySavannas_Mask/'
evergreenbroad_mask_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/EvergreenBroad_Mask/'
deciduousbroad_mask_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/DeciduousBroad_Mask/'
openshrublands_mask_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/OpenShrublands_Mask/'
closedshrublands_mask_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/ClosedShrublands_Mask/'
barren_mask_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/MCD12C1/Barren_Mask/'

grasslands_masked_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover/Variables_Grasslands/'
croplands_masked_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover/Variables_Croplands/'
cropnatveg_masked_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover/Variables_CropNatVeg/'
savannas_masked_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover/Variables_Savannas/' 
woodysavannas_masked_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover/Variables_WoodySavannas/'
evergreenbroad_masked_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover/Variables_EvergreenBroad/'
deciduousbroad_masked_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover/Variables_DeciduousBroad/'
openshrublands_masked_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover/Variables_OpenShrublands/'
closedshrublands_masked_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover/Variables_ClosedShrublands/'
barren_masked_out_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover/Variables_Barren/'

all_dirs = ['/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/CHIRPS/Resampled/', '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/CHIRTS/Dekads/', '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/Resampled/', '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS_SM/Dekads/', '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/Hobbins_ET/Resampled/', '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/LST/Resampled/']

# Multiply a chosen land cover mask by all parameters for each year

In [ ]:
def rastercalc_lcmask(lcmask_dir, param_dirs, out_dir):
    """
    This function iterates through all parameter files and multiplies the land cover mask for a certain year
    to create output rasters of each parameter masked by land cover type
    Note: must change the out name of the land cover in the out file name 
    """
    lcfiles=np.array(sorted(os.listdir(lcmask_dir)))
    
    
    for yr in range(2002,2017):
        lc_yr = [file for file in lcfiles if str(yr) in file]
        for lc in lc_yr:
            lc_mask = rasterio.open(lcmask_dir + lc)
            for folder in param_dirs:
                var_yr = [file for file in sorted(os.listdir(folder)) if str(yr) in file]
                for var in var_yr:
                    var_rast = rasterio.open(folder+var)
                    meta = var_rast.meta
                    var_lcmask = var_rast.read(1)*lc_mask.read(1)
                    with rasterio.open(out_dir + 'evergreenbroad_' + var, 'w', **meta) as dst:
                            dst.write(var_lcmask, 1)
                            print(var)

In [ ]:
rastercalc_lcmask(evergreenbroad_mask_dir, all_dirs, evergreenbroad_masked_out_dir)

# Rasterstats with Admin Boundaries

In [3]:
def zone_stat(raster, band, polygon, stats):
    """
    This function will calculate the zonal stats for each polygon within a raster
    requires gpd_df, raster, object and nodata value
    
    Args: raster = input raster
          band = band of raster
          polygon = polygons to calc zonal stats 
          stats = stat to calculate as string
    """
    
    band = raster.read(band)
    zone_stat = zonal_stats(polygon, band, affine=raster.meta['transform'], nodata = np.nan, stats = stats)
    return zone_stat

In [4]:
def var_poly_join(in_dir, gdf):
    files=np.array(sorted(os.listdir(in_dir)))
    tifs = pd.Series(files).str.contains('.tif')
    files = files[tifs]
    
    for filename in files:
        print(filename)
        raster = rasterio.open(in_dir+filename)
        stats = zone_stat(raster, 1, gdf, 'mean')
        print((list(stats))[-1])
        name = os.path.splitext(os.path.basename(filename))[0]
        gdf['Mean'+ "_" + name] = gpd.GeoDataFrame.from_dict(stats)
    return gdf

In [5]:
lcmasked_in_dir = croplands_masked_out_dir 

In [6]:
adminbds = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/AdminBoundaries/Africa_zones_2019/g2008_af_1.shp'
bndry = gpd.read_file(adminbds)

ea_adminbds = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/AdminBoundaries/gadm36_EastAfrica.shp'
ea_bndry = gpd.read_file(ea_adminbds)

oromia = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/AdminBoundaries/Ethiopia/oromia.shp'
oromia_bndry = gpd.read_file(oromia)

In [ ]:
output_shp = var_poly_join(lcmasked_in_dir, ea_bndry)

croplands_SM_200207_01.tif
{'mean': None}
croplands_SM_200207_02.tif
{'mean': None}
croplands_SM_200207_03.tif
{'mean': None}
croplands_SM_200208_01.tif
{'mean': None}
croplands_SM_200208_02.tif
{'mean': None}
croplands_SM_200208_03.tif
{'mean': None}
croplands_SM_200209_01.tif
{'mean': None}
croplands_SM_200209_02.tif
{'mean': None}
croplands_SM_200209_03.tif
{'mean': None}
croplands_SM_200210_01.tif
{'mean': None}
croplands_SM_200210_02.tif
{'mean': None}
croplands_SM_200210_03.tif
{'mean': None}
croplands_SM_200211_01.tif
{'mean': None}
croplands_SM_200211_02.tif
{'mean': None}
croplands_SM_200211_03.tif
{'mean': None}
croplands_SM_200212_01.tif
{'mean': None}
croplands_SM_200212_02.tif
{'mean': None}
croplands_SM_200212_03.tif
{'mean': None}
croplands_SM_200301_01.tif
{'mean': None}
croplands_SM_200301_02.tif
{'mean': None}
croplands_SM_200301_03.tif
{'mean': None}
croplands_SM_200302_01.tif
{'mean': None}
croplands_SM_200302_02.tif
{'mean': None}
croplands_SM_200302_03.tif
{'mean'

{'mean': 0.3511718511581421}
croplands_SM_200706_02.tif
{'mean': 0.36449193954467773}
croplands_SM_200706_03.tif
{'mean': 0.3511340320110321}
croplands_SM_200707_01.tif
{'mean': 0.36208948493003845}
croplands_SM_200707_02.tif
{'mean': 0.349590927362442}
croplands_SM_200707_03.tif
{'mean': 0.3513888120651245}
croplands_SM_200708_01.tif
{'mean': 0.37837058305740356}
croplands_SM_200708_02.tif
{'mean': 0.39937540888786316}
croplands_SM_200708_03.tif
{'mean': 0.399065226316452}
croplands_SM_200709_01.tif
{'mean': 0.39582017064094543}
croplands_SM_200709_02.tif
{'mean': 0.3867732584476471}
croplands_SM_200709_03.tif
{'mean': 0.37757378816604614}
croplands_SM_200710_01.tif
{'mean': 0.3842085599899292}
croplands_SM_200710_02.tif
{'mean': 0.3729272484779358}
croplands_SM_200710_03.tif
{'mean': 0.3837195336818695}
croplands_SM_200711_01.tif
{'mean': 0.3735411465167999}
croplands_SM_200711_02.tif
{'mean': 0.3722425103187561}
croplands_SM_200711_03.tif
{'mean': 0.3139917552471161}
croplands_SM_20

{'mean': 0.37922394275665283}
croplands_SM_201106_03.tif
{'mean': 0.3838312327861786}
croplands_SM_201107_01.tif
{'mean': 0.39512529969215393}
croplands_SM_201107_02.tif
{'mean': 0.3951921761035919}
croplands_SM_201107_03.tif
{'mean': 0.400427907705307}
croplands_SM_201108_01.tif
{'mean': 0.4061582684516907}
croplands_SM_201108_02.tif
{'mean': 0.40730366110801697}
croplands_SM_201108_03.tif
{'mean': 0.41173890233039856}
croplands_SM_201109_01.tif
{'mean': 0.425794780254364}
croplands_SM_201109_02.tif
{'mean': 0.41871610283851624}
croplands_SM_201109_03.tif
{'mean': 0.41519418358802795}
croplands_SM_201110_01.tif
{'mean': 0.4192017614841461}
croplands_SM_201110_02.tif
{'mean': 0.42158636450767517}
croplands_SM_201110_03.tif
{'mean': 0.4072472155094147}
croplands_SM_201111_01.tif
{'mean': 0.4210684299468994}
croplands_SM_201111_02.tif
{'mean': 0.41877850890159607}
croplands_SM_201111_03.tif
{'mean': 0.4078757166862488}
croplands_SM_201112_01.tif
{'mean': 0.38687020540237427}
croplands_SM

{'mean': 0.3908902704715729}
croplands_SM_201507_02.tif
{'mean': 0.37433576583862305}
croplands_SM_201507_03.tif
{'mean': 0.3985694646835327}
croplands_SM_201508_01.tif
{'mean': 0.4005991518497467}
croplands_SM_201508_02.tif
{'mean': 0.38978311419487}
croplands_SM_201508_03.tif
{'mean': 0.3952506184577942}
croplands_SM_201509_01.tif
{'mean': 0.39279860258102417}
croplands_SM_201509_02.tif
{'mean': 0.3923410475254059}
croplands_SM_201509_03.tif
{'mean': 0.391361802816391}
croplands_SM_201510_01.tif
{'mean': 0.3914981782436371}
croplands_SM_201510_02.tif
{'mean': 0.40658292174339294}
croplands_SM_201510_03.tif
{'mean': 0.4197447896003723}
croplands_SM_201511_01.tif
{'mean': 0.4233326315879822}
croplands_SM_201511_02.tif
{'mean': 0.4226751923561096}
croplands_SM_201511_03.tif
{'mean': 0.41293865442276}
croplands_SM_201512_01.tif
{'mean': 0.40975266695022583}
croplands_SM_201512_02.tif
{'mean': 0.3839832544326782}
croplands_SM_201512_03.tif
{'mean': 0.3538511395454407}
croplands_SM_201601_

{'mean': None}
croplands_Tmax_200510_03.tif
{'mean': None}
croplands_Tmax_200511_01.tif
{'mean': None}
croplands_Tmax_200511_02.tif
{'mean': None}
croplands_Tmax_200511_03.tif
{'mean': None}
croplands_Tmax_200512_01.tif
{'mean': None}
croplands_Tmax_200512_02.tif
{'mean': None}
croplands_Tmax_200512_03.tif
{'mean': None}
croplands_Tmax_200601_01.tif
{'mean': 35.652626037597656}
croplands_Tmax_200601_02.tif
{'mean': 33.449005126953125}
croplands_Tmax_200601_03.tif
{'mean': 35.88713836669922}
croplands_Tmax_200602_01.tif
{'mean': 37.58643341064453}
croplands_Tmax_200602_02.tif
{'mean': 35.10908889770508}
croplands_Tmax_200602_03.tif
{'mean': 34.207733154296875}
croplands_Tmax_200603_01.tif
{'mean': 33.115562438964844}
croplands_Tmax_200603_02.tif
{'mean': 32.81608581542969}
croplands_Tmax_200603_03.tif
{'mean': 32.438716888427734}
croplands_Tmax_200604_01.tif
{'mean': 32.960289001464844}
croplands_Tmax_200604_02.tif
{'mean': 32.074562072753906}
croplands_Tmax_200604_03.tif
{'mean': 32.08

...

{'mean': 30.989389419555664}
croplands_rs_lst.2015.072.tif
{'mean': 34.366172790527344}
croplands_rs_lst.2015.073.tif
{'mean': 31.24928092956543}
croplands_rs_lst.2015.081.tif
{'mean': 30.0185604095459}
croplands_rs_lst.2015.082.tif
{'mean': 33.86735153198242}
croplands_rs_lst.2015.083.tif
{'mean': 34.29092025756836}
croplands_rs_lst.2015.091.tif
{'mean': 31.75501823425293}
croplands_rs_lst.2015.092.tif
{'mean': 32.23356628417969}
croplands_rs_lst.2015.093.tif
{'mean': 32.799129486083984}
croplands_rs_lst.2015.101.tif
{'mean': 31.063817977905273}
croplands_rs_lst.2015.102.tif
{'mean': 29.565013885498047}
croplands_rs_lst.2015.103.tif
{'mean': 25.898754119873047}
croplands_rs_lst.2015.111.tif
{'mean': 29.33761215209961}
croplands_rs_lst.2015.112.tif
{'mean': 28.48432159423828}
croplands_rs_lst.2015.113.tif
{'mean': 30.063858032226562}
croplands_rs_lst.2015.121.tif
{'mean': 31.074277877807617}
croplands_rs_lst.2015.122.tif
{'mean': 33.294288635253906}
croplands_rs_lst.2015.123.tif
{'mean

{'mean': None}
croplands_rs_ndvi.2005.081.tif
{'mean': None}
croplands_rs_ndvi.2005.082.tif
{'mean': None}
croplands_rs_ndvi.2005.083.tif
{'mean': None}
croplands_rs_ndvi.2005.091.tif
{'mean': None}
croplands_rs_ndvi.2005.092.tif
{'mean': None}
croplands_rs_ndvi.2005.093.tif
{'mean': None}
croplands_rs_ndvi.2005.101.tif
{'mean': None}
croplands_rs_ndvi.2005.102.tif
{'mean': None}
croplands_rs_ndvi.2005.103.tif
{'mean': None}
croplands_rs_ndvi.2005.111.tif
{'mean': None}
croplands_rs_ndvi.2005.112.tif
{'mean': None}
croplands_rs_ndvi.2005.113.tif
{'mean': None}
croplands_rs_ndvi.2005.121.tif
{'mean': None}
croplands_rs_ndvi.2005.122.tif
{'mean': None}
croplands_rs_ndvi.2005.123.tif
{'mean': None}
croplands_rs_ndvi.2006.011.tif
{'mean': 142.2510986328125}
croplands_rs_ndvi.2006.012.tif
{'mean': 139.4358367919922}
croplands_rs_ndvi.2006.013.tif
{'mean': 137.95176696777344}
croplands_rs_ndvi.2006.021.tif
{'mean': 136.29115295410156}
croplands_rs_ndvi.2006.022.tif
{'mean': 134.8781585693359

{'mean': 163.913330078125}
croplands_rs_ndvi.2009.072.tif
{'mean': 163.2168426513672}
croplands_rs_ndvi.2009.073.tif
{'mean': 163.52114868164062}
croplands_rs_ndvi.2009.081.tif
{'mean': 164.9757843017578}
croplands_rs_ndvi.2009.082.tif
{'mean': 166.87010192871094}
croplands_rs_ndvi.2009.083.tif
{'mean': 168.64581298828125}
croplands_rs_ndvi.2009.091.tif
{'mean': 170.23968505859375}
croplands_rs_ndvi.2009.092.tif
{'mean': 171.89466857910156}
croplands_rs_ndvi.2009.093.tif
{'mean': 173.06497192382812}
croplands_rs_ndvi.2009.101.tif
{'mean': 172.76287841796875}
croplands_rs_ndvi.2009.102.tif
{'mean': 172.36883544921875}
croplands_rs_ndvi.2009.103.tif
{'mean': 172.1544647216797}
croplands_rs_ndvi.2009.111.tif
{'mean': 171.85879516601562}
croplands_rs_ndvi.2009.112.tif
{'mean': 170.83111572265625}
croplands_rs_ndvi.2009.113.tif
{'mean': 169.47842407226562}
croplands_rs_ndvi.2009.121.tif
{'mean': 167.41323852539062}
croplands_rs_ndvi.2009.122.tif
{'mean': 164.63870239257812}
croplands_rs_ndv

{'mean': 166.12307739257812}
croplands_rs_ndvi.2013.052.tif
{'mean': 167.44952392578125}
croplands_rs_ndvi.2013.053.tif
{'mean': 167.58453369140625}
croplands_rs_ndvi.2013.061.tif
{'mean': 167.85885620117188}
croplands_rs_ndvi.2013.062.tif
{'mean': 168.68722534179688}
croplands_rs_ndvi.2013.063.tif
{'mean': 169.6201171875}
croplands_rs_ndvi.2013.071.tif
{'mean': 170.49713134765625}
croplands_rs_ndvi.2013.072.tif
{'mean': 170.33526611328125}
croplands_rs_ndvi.2013.073.tif
{'mean': 170.83529663085938}
croplands_rs_ndvi.2013.081.tif
{'mean': 171.62599182128906}
croplands_rs_ndvi.2013.082.tif
{'mean': 172.70672607421875}
croplands_rs_ndvi.2013.083.tif
{'mean': 172.7969207763672}
croplands_rs_ndvi.2013.091.tif
{'mean': 172.80633544921875}
croplands_rs_ndvi.2013.092.tif
{'mean': 173.08143615722656}
croplands_rs_ndvi.2013.093.tif
{'mean': 174.64817810058594}
croplands_rs_ndvi.2013.101.tif
{'mean': 176.31552124023438}
croplands_rs_ndvi.2013.102.tif
{'mean': 176.46221923828125}
croplands_rs_ndv

{'mean': None}
croplands_rs_pet_200229.tif
{'mean': None}
croplands_rs_pet_200230.tif
{'mean': None}
croplands_rs_pet_200231.tif
{'mean': None}
croplands_rs_pet_200232.tif
{'mean': None}
croplands_rs_pet_200233.tif
{'mean': None}
croplands_rs_pet_200234.tif
{'mean': None}
croplands_rs_pet_200235.tif
{'mean': None}
croplands_rs_pet_200236.tif
{'mean': None}
croplands_rs_pet_200301.tif
{'mean': None}
croplands_rs_pet_200302.tif
{'mean': None}
croplands_rs_pet_200303.tif
{'mean': None}
croplands_rs_pet_200304.tif
{'mean': None}
croplands_rs_pet_200305.tif
{'mean': None}
croplands_rs_pet_200306.tif
{'mean': None}
croplands_rs_pet_200307.tif
{'mean': None}
croplands_rs_pet_200308.tif
{'mean': None}
croplands_rs_pet_200309.tif
{'mean': None}
croplands_rs_pet_200310.tif
{'mean': None}
croplands_rs_pet_200311.tif
{'mean': None}
croplands_rs_pet_200312.tif
{'mean': None}
croplands_rs_pet_200313.tif
{'mean': None}
croplands_rs_pet_200314.tif
{'mean': None}
croplands_rs_pet_200315.tif
{'mean': No

In [ ]:
output_df = pd.DataFrame(output_shp.drop(columns='geometry'))

In [ ]:
output_df.tail()

In [ ]:
output_df.shape

In [ ]:
output_df.to_csv('/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/Final_TS/'+'ea_closedshrublands.csv')



# END

# Reassign cloud mask value before calculating stats

In [ ]:
def reassign_masks(in_dir):
    files=np.array(sorted(os.listdir(in_dir)))
    tifs = pd.Series(files).str.contains('.tif')
    files = files[tifs]

    for filename in files:
        with rasterio.open(in_dir+filename, 'r+') as ds:
            print(filename)
            a = ds.read()# read all raster values
            a[a < 0 ] = np.nan  
            ds.write(a)

In [ ]:
reassign_masks(variables_mask_in_dir)

# Trying dask delayed

In [ ]:
@dask.delayed
def read_and_mean(filename, in_dir, admin_gdf):
    """
    input: 
    in_dir - directory of masked land cover type
    filename - a raster variable for particular dekad
    admin_gdf - administrative boundary layer of Africa as geodataframe
    This function opens each raster and extract zonal mean for each variable
    of a particular land cover type
    """
    print(filename)
    raster = rasterio.open(in_dir+filename)
    mean = zone_stat(raster, 1, admin_gdf, 'mean')
    name = os.path.splitext(os.path.basename(filename))[0]
    return {'Mean'+ "_" + name : mean}

In [ ]:
files=np.array(sorted(os.listdir(in_dir)))
tifs = pd.Series(files).str.contains('.tif')
files = files[tifs]

admin_mean_list = []
for filename in files:
    zmeans = read_and_mean(filename, in_dir, bndry)
    admin_mean_list.append(zmeans)


In [ ]:
np.nanx

In [ ]:
from dask.distributed import Client
client = Client()
client

In [ ]:
client.compute(admin_mean_list,scheduler = 'processes')

In [ ]:
#var_grass_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/Variables_Croplands/'
#year_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/Variables_Grasslands/2016/'

In [ ]:
#output_df = output_df[output_df.columns.drop(list(output_df.filter(regex='2014')))] 
#if you want to reverse and remove a specific year from the appended dataframe

# Remove nan pixels from lst

In [ ]:
lst_test = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover/Variables_Clos/Version2/cropland_rs_chirps-v2-Copy1.0.2002.07.1.tif'

In [ ]:
lst_array = lst_test.read(1)

In [ ]:
lst_array.min()

In [ ]:
with rasterio.open(lst_test, 'r+') as ds:
    a = ds.read()# read all raster values
    #lst_array[np.where(lst_array<=0)]
    a[a < 0] = 0  #set all values not cropland as 0
    ds.write(a)

In [ ]:
lst_test2 = rasterio.open(lst_test)
lst_arr = lst_test2.read()
lst_arr.min()

In [ ]:
len(lst_arr[np.where(lst_arr<0)])

In [ ]:
show(lst_test2)

In [ ]:
lst_mask = lst_test.read_masks(1)

In [ ]:
lst_mask

In [ ]:
lst_array.shape

In [ ]:
lst_array[np.where(lst_array<=0)]

In [ ]:
np.where(lst_array<=0)

In [ ]:
with rasterio.open(lc_mask, 'r+') as ds:
    a = ds.read()  # read all raster values
    a[a!=4] = 0  #set all values not cropland as 0
    a[a==4] = 1
    ds.write(a)

In [ ]:
def create_land_mask(in_dir, out_dir, lc_num):
    files=np.array(sorted(os.listdir(in_dir)))
    
    for filename in files:
        base = os.path.splitext(filename)[0]
        print(base)
        with rasterio.open(in_dir+filename, 'r+') as ds:
            rast = ds.read()
            rast[rast!=lc_num] = np.nan
            rast[rast==lc_num] = 1
            meta = ds.meta()
            meta.update(
                dtype='float32')
            ds.write(rast)
                
                
                
#         rast = rasterio.open(in_dir+filename)
#         meta = rast.meta
#         meta.update(
#             dtype='float32')
#         #rast.close()
#         print(meta)
   
        
#         with rasterio.open(out_dir+base+'_' + str(lc_num) + '.tif', 'w', **meta) as dst:
#             #with rasterio.open(in_dir+filename) as src:
#                 data = rast.read()
#                 data[data!=lc_num] = -9999
#                 data[data==lc_num] = 1
#                 dst.write(data)